<a href="https://colab.research.google.com/github/linhoang04/ML_Hoc_sau/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_QNN_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 90.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


In [ ]:
df = pd.read_csv('https://github.com/urwithajit9/ClaMP/raw/master/dataset/ClaMP_Integrated-5184.csv', sep=',')

In [ ]:
df.drop(['packer_type'],axis=1,inplace=True)

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)

In [ ]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [ ]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [ ]:
df.drop(['index'],axis=1,inplace=True)
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,248,4,1,1,1,...,1,2,2,0,0.000000,0.255780,118784,5.489693,0,1
1,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.373499,3.530117,206336,6.029231,1,0
2,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.300261,4.488365,208944,6.313596,1,0
3,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.428835,5.817769,173952,6.396898,1,0
4,144,3,4,65535,184,240,4,1,1,1,...,1,3,1,0,0.000000,0.000000,118784,5.636154,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,240,4,1,1,1,...,1,2,2,0,3.465415,1.282154,102400,5.700827,0,1
5206,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,0,0.000000,0.000000,283136,7.850822,0,1
5207,144,3,4,65535,184,256,3,1,1,1,...,1,2,1,1,0.000000,0.000000,151040,7.974994,1,1
5208,144,3,4,65535,184,240,7,1,1,1,...,1,4,3,0,0.000000,5.036332,191488,5.176947,1,1


In [ ]:

# phan tram do du lieu (5210 là 100%)
df_5percent=df[0:5210]


df_5percent



,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,248,4,1,1,1,...,1,2,2,0,0.000000,0.255780,118784,5.489693,0,1
1,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.373499,3.530117,206336,6.029231,1,0
2,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.300261,4.488365,208944,6.313596,1,0
3,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.428835,5.817769,173952,6.396898,1,0
4,144,3,4,65535,184,240,4,1,1,1,...,1,3,1,0,0.000000,0.000000,118784,5.636154,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,240,4,1,1,1,...,1,2,2,0,3.465415,1.282154,102400,5.700827,0,1
5206,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,0,0.000000,0.000000,283136,7.850822,0,1
5207,144,3,4,65535,184,256,3,1,1,1,...,1,2,1,1,0.000000,0.000000,151040,7.974994,1,1
5208,144,3,4,65535,184,240,7,1,1,1,...,1,4,3,0,0.000000,5.036332,191488,5.176947,1,1


In [ ]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [ ]:
df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

df_5percent.info



<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     248.0               4.0   
1      144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
2      144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
3      144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
4      144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
...      ...   ...        ...         ...    ...       ...               ...   
5205   144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
5206    80.0   2.0        4.0     65535.0  184.0     256.0               8.0   
5207   144.0   3.0        4.0     65535.0  184.0     256.0               3.0   
5208   144.0   3.0        4.0     65535.0  184.0     240.0               7.0   
5209   144.0   3.0        4.0     65535.0  184.0     200.0               4.0   

      CreationYear  FH_char0  FH_char1  ...  LoaderFlags  sus_sections  \
0              1.0       1.0       1.0  ...          1.0           2.0   
1              1.0       0.0       1.0  ...          1.0           1.0   
2              1.0       0.0       1.0  ...          1.0           1.0   
3              1.0       0.0       1.0  ...          1.0           1.0   
4              1.0       1.0       1.0  ...          1.0           3.0   
...            ...       ...       ...  ...          ...           ...   
5205           1.0       1.0       1.0  ...          1.0           2.0   
5206           1.0       0.0       1.0  ...          1.0           4.0   
5207           1.0       1.0       1.0  ...          1.0           2.0   
5208           1.0       1.0       1.0  ...          1.0           4.0   
5209           1.0       1.0       1.0  ...          1.0           0.0   

      non_sus_sections  packer    E_text    E_data  filesize    E_file  \
0                  2.0     0.0  0.000000  0.255780  118784.0  5.489693   
1                  4.0     0.0  6.373499  3.530117  206336.0  6.029231   
2                  3.0     0.0  6.300261  4.488365  208944.0  6.313596   
3                  3.0     0.0  6.428835  5.817769  173952.0  6.396898   
4                  1.0     0.0  0.000000  0.000000  118784.0  5.636154   
...                ...     ...       ...       ...       ...       ...   
5205               2.0     0.0  3.465415  1.282154  102400.0  5.700827   
5206               4.0     0.0  0.000000  0.000000  283136.0  7.850822   
5207               1.0     1.0  0.000000  0.000000  151040.0  7.974994   
5208               3.0     0.0  0.000000  5.036332  191488.0  5.176947   
5209               4.0     0.0  6.222104  7.153278   67584.0  6.937505   

      fileinfo  class  
0          0.0    1.0  
1          1.0    0.0  
2          1.0    0.0  
3          1.0    0.0  
4          0.0    1.0  
...        ...    ...  
5205       0.0    1.0  
5206       0.0    1.0  
5207       1.0    1.0  
5208       1.0    1.0  
5209       0.0    1.0  

[5210 rows x 69 columns]>

In [ ]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.0,...,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5210.000000,5.210000e+03,5210.000000,5210.000000,5210.000000
mean,152.658733,10.633589,8.903263,65137.618234,202.530902,222.833013,4.662956,0.990403,0.362764,1.0,...,0.999040,1.360845,3.302111,0.156430,4.939586,2.523549,7.875718e+05,6.364756,0.544146,0.522457
std,616.499070,392.745064,251.685103,5005.606042,968.625098,48.591497,1.922882,0.097502,0.480844,0.0,...,0.030967,1.623972,1.146956,0.363297,2.521787,2.685909,5.293440e+06,1.119881,0.498095,0.499543
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.886006,0.000000,6.144000e+04,5.695785,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.148211,1.555973,1.218560e+05,6.390618,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.507072,4.685836,3.055480e+05,7.314531,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,65534.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999859,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [ ]:

# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [ ]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 4
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [ ]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [ ]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

,proportion
class,
1.0,51.875
0.0,48.125


In [ ]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)

    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')

    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))

    return qml.expval(qml.PauliZ(0))

In [ ]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[-0.00774043 -0.00359778  0.00724083]
  [-0.00255765  0.00849921 -0.01311324]
  [-0.00870305 -0.00506643 -0.01309951]
  [ 0.02943663 -0.01096266  0.00914884]]

 [[-0.00666061 -0.00513787  0.00301323]
  [-0.01458517 -0.00662738 -0.00146473]
  [-0.00846017 -0.00822489  0.01086727]
  [ 0.01004987 -0.00441766 -0.00084165]]

 [[ 0.01237016 -0.01426479  0.00338023]
  [ 0.01974571  0.01965035 -0.01939911]
  [-0.00762627  0.00168659 -0.01321877]
  [-0.00668247 -0.00137291  0.01401322]]

 [[-0.00045403  0.00216937  0.00512424]
  [ 0.00543489  0.00028099 -0.01197087]
  [ 0.00789673  0.00431489 -0.01096752]
  [-0.00682302  0.00887512  0.00605483]]

 [[-0.00720298 -0.00269934 -0.00143462]
  [ 0.00617004  0.01248941  0.00065649]
  [ 0.01169208  0.00873317  0.0020565 ]
  [-0.01443483 -0.01027711  0.00741241]]] 0.0


In [ ]:
circuit(weights_init, data[0])

tensor(-0.01125409, requires_grad=True)

In [ ]:

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [ ]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [ ]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [ ]:

def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [ ]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.29135833, requires_grad=False), tensor(-0.19509512, requires_grad=False), tensor(0.02707174, requires_grad=False), tensor(-0.17403743, requires_grad=False)], Y =  1
X = [tensor(-0.29320212, requires_grad=False), tensor(-0.12851595, requires_grad=False), tensor(0.03000398, requires_grad=False), tensor(-0.22193397, requires_grad=False)], Y = -1
X = [tensor(-0.2918126, requires_grad=False), tensor(-0.17607146, requires_grad=False), tensor(0.02793881, requires_grad=False), tensor(-0.19601437, requires_grad=False)], Y =  1
X = [tensor(-0.29175443, requires_grad=False), tensor(-0.17490805, requires_grad=False), tensor(0.02799209, requires_grad=False), tensor(-0.47296723, requires_grad=False)], Y =  1
X = [tensor(-0.20821671, requires_grad=False), tensor(-0.0553551, requires_grad=False), tensor(0.2561202, requires_grad=False), tensor(0.69691405, requires_grad=False)], Y =  1


In [ ]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [ ]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1167868 | Accuracy: 0.4812500 
New best
Iter:     2 | Cost: 1.0209825 | Accuracy: 0.5062500 
New best
Iter:     3 | Cost: 0.9924139 | Accuracy: 0.5375000 
New best
Iter:     4 | Cost: 1.0270889 | Accuracy: 0.5562500 
Iter:     5 | Cost: 1.1222849 | Accuracy: 0.5312500 
Iter:     6 | Cost: 1.0894430 | Accuracy: 0.5062500 
Iter:     7 | Cost: 1.0053176 | Accuracy: 0.5250000 
Iter:     8 | Cost: 0.9859319 | Accuracy: 0.5250000 
Iter:     9 | Cost: 0.9809744 | Accuracy: 0.5125000 
Iter:    10 | Cost: 0.9709903 | Accuracy: 0.5187500 
Iter:    11 | Cost: 0.9658364 | Accuracy: 0.5125000 
Iter:    12 | Cost: 0.9670892 | Accuracy: 0.5500000 
New best
Iter:    13 | Cost: 0.9674507 | Accuracy: 0.6000000 
Iter:    14 | Cost: 0.9646419 | Accuracy: 0.5250000 
Iter:    15 | Cost: 0.9683871 | Accuracy: 0.5125000 
Iter:    16 | Cost: 0.9699362 | Accuracy: 0.5062500 
Iter:    17 | Cost: 0.9718058 | Accuracy: 0.5375000 
Iter:    18 | Cost: 0.9813237 | Accuracy: 0.5500000 
I

In [ ]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [ ]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

In [ ]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

In [ ]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))